# Transformaciones de Datos para Métricas de Atención al Cliente en ACME

Este notebook ha sido diseñado para realizar transformaciones sobre los datos disponibles en el Datalake de ACME, con el fin de preparar un conjunto de tablas que serán utilizadas por el equipo de BI para el desarrollo de un tablero de control. La meta principal de estas transformaciones es dar seguimiento y análisis a las métricas clave relacionadas con el sector de atención al cliente.

### Justificación del Enfoque
En este challenge, opté por utilizar Jupyter Notebook para realizar las transformaciones y consultas requeridas. La elección de este enfoque se basó más que nada en demostrar el proceso de forma paso a paso y mantener la transparencia del análisis. Decidí trabajar con los datos de ejemplo proporcionados y cargarlos en  DataFrames que simulan las tablas. Luego, los datos fueron manipulados utilizando SQLite para llevar a cabo las consultas y transformaciones necesarias.

In [2]:
# Importar bibliotecas necesarias
import pandas as pd
import sqlite3
from IPython.display import display

#### 1. Crear DataFrames con los datos de ejemplo

In [3]:
# Crear el DataFrame de interacciones
df_interactions = pd.DataFrame({
    'case_id': [32, 32, 32, 32, 32, 32, 32, 32, 32],
    'interaction_id': [1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140],
    'interaction_type': ['user_contact', 'rep_response', 'user_recontact', 'rep_response',
                         'rep_derivation', 'rep_response', 'rep_response', 'rep_case_closed', 'survey_sent'],
    'representante': [None, 'ggutierrez', None, 'jperez', 'jperez', 'pcruz', 'pcruz', 'pcruz', 'pcruz'],
    'int_date': ['01/02/2020 10:54', '01/02/2020 11:45', '02/02/2020 9:45', '02/02/2020 12:35',
                 '02/02/2020 12:39', '02/02/2020 20:39', '02/02/2020 20:41', '02/02/2020 20:42', None]
})

# Crear el DataFrame de representantes
df_representantes = pd.DataFrame({
    'representante': ['ggutierrez', 'jperez', 'pcruz'],
    'full_name': ['Gastón Gutierrez', 'Juan Pérez', 'Pablo Cruz'],
    'team': [1, 2, 2],
    'incoming_date': ['01/01/2027', '05/10/2019', '01/01/2020'],
    'status': ['Active', 'Active', 'Active']
})

# Crear el DataFrame de encuestas
df_surveys = pd.DataFrame({
    'survey_id': [2, 3, 4, 5, 6],
    'case_id': [32, 41, 59, 145, 234],
    'nps_score': [9, 1, 8, 7, 10],
    'nps_comment': ['!Muy buen atención!', 'Tardaron mucho en responderme', 'Gracias por su respuesta!',
                    None, 'Todo muy bien']
})

# Convertir columnas de fechas a tipo datetime
df_interactions['int_date'] = pd.to_datetime(df_interactions['int_date'], errors='coerce')
df_representantes['incoming_date'] = pd.to_datetime(df_representantes['incoming_date'], errors='coerce')


#### 2. Conexión a SQLite en memoria y carga de los datos

In [4]:
conn = sqlite3.connect(':memory:')  # Conexión a SQLite en memoria

In [5]:
# Cargar DataFrames en la base de datos SQLite
df_interactions.to_sql('interactions', conn, index=False, if_exists='replace')
df_representantes.to_sql('representantes', conn, index=False, if_exists='replace')
df_surveys.to_sql('surveys', conn, index=False, if_exists='replace')


5

### Consigna 1
#### a. Consulta SQL para Calcular el NPS de Casos Derivados por Mes

Esta consulta SQL calcula el Net Promoter Score (NPS) para los casos que han sido derivados al menos una vez. El NPS se calcula considerando solo las interacciones donde el caso ha sido derivado o cerrado por un representante.


In [6]:
query_derivados = """
SELECT
    strftime('%Y-%m', i.int_date) AS month,
    COUNT(DISTINCT CASE WHEN i.interaction_type = 'rep_derivation' THEN i.case_id END) AS casos_derivados,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) AS cantidad_promotores,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END) AS cantidad_detractores,
    COUNT(DISTINCT s.case_id) AS total_encuestas,
    ((COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) -
      COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END)) * 1.0 /
     COUNT(DISTINCT s.case_id)) * 100 AS nps
FROM
    interactions i
LEFT JOIN
    surveys s ON i.case_id = s.case_id
WHERE
    i.interaction_type IN ('rep_derivation', 'rep_case_closed')
GROUP BY
    month;
"""

# Execute query and display the result
df_nps_derivados = pd.read_sql(query_derivados, conn)
df_nps_derivados


,month,casos_derivados,cantidad_promotores,cantidad_detractores,total_encuestas,nps
0,2020-02,1,1,0,1,100.0


##### Descripción de la Consulta:

1. **Agrupación por Mes**:
   La consulta utiliza la función `strftime('%Y-%m', i.int_date)` para extraer el mes y año de la fecha de la interacción y agrupar los resultados por mes.

2. **Cálculo de Casos Derivados**:
   Se cuenta el número de casos que tienen el tipo de interacción `'rep_derivation'` usando `COUNT(DISTINCT CASE WHEN i.interaction_type = 'rep_derivation' THEN i.case_id END)`.

3. **Cálculo de Promotores y Detractores**:
   Se cuentan los casos que tienen un puntaje de NPS entre 9 y 10 como "promotores" y los casos con un puntaje entre 0 y 6 como "detractores".

4. **Cálculo de Total de Encuestas**:
   Se calcula el total de encuestas realizadas usando `COUNT(DISTINCT s.case_id)`.

5. **Cálculo del NPS**:
   El NPS se calcula con la fórmula: 
   \[
   \text{NPS} = \left( \frac{\text{Cantidad de Promotores} - \text{Cantidad de Detractores}}{\text{Total de Encuestas}} \right) \times 100
   \]
   Esta fórmula convierte el NPS en un porcentaje que representa la diferencia entre la cantidad de promotores y detractores.


#### b. Consulta SQL para Calcular el NPS de Casos que no tuvieron ninguna derivación

In [7]:
query_no_derivados = """
SELECT
    strftime('%Y-%m', i.int_date) AS month,
    COUNT(DISTINCT CASE WHEN i.interaction_type = 'rep_derivation' THEN i.case_id END) AS casos_sin_derivacion,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) AS cantidad_promotores,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END) AS cantidad_detractores,
    COUNT(DISTINCT s.case_id) AS total_encuestas,
    ((COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) -
      COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END)) * 1.0 /
     COUNT(DISTINCT s.case_id)) * 100 AS nps
FROM
    interactions i
LEFT JOIN
    surveys s ON i.case_id = s.case_id
WHERE
    i.interaction_type = 'user_contact'
    AND NOT EXISTS (
        SELECT 1
        FROM interactions i2
        WHERE i2.case_id = i.case_id
          AND i2.interaction_type = 'rep_derivation'
    )
GROUP BY
    month;
"""

# Execute query and display the result
df_nps_no_derivados = pd.read_sql(query_no_derivados, conn)
df_nps_no_derivados


,month,casos_sin_derivacion,cantidad_promotores,cantidad_detractores,total_encuestas,nps


### Consigna 2
#### Se requiere visualizar el NPS por cada equipo de representantes y por mes.

In [8]:
# Consulta SQL para obtener el NPS por equipo y por mes
query_nps_por_equipo = """
SELECT
    strftime('%Y-%m', i.int_date) AS month,
    r.team,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) AS cantidad_promotores,
    COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END) AS cantidad_detractores,
    COUNT(DISTINCT s.case_id) AS total_encuestas,
    ((COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 9 AND 10 THEN s.case_id END) - 
      COUNT(DISTINCT CASE WHEN s.nps_score BETWEEN 0 AND 6 THEN s.case_id END)) * 1.0 / 
     COUNT(DISTINCT s.case_id)) * 100 AS nps
FROM
    interactions i
JOIN
    representantes r ON i.representante = r.representante
LEFT JOIN
    surveys s ON i.case_id = s.case_id
WHERE
    i.interaction_type IN ('rep_response', 'rep_derivation', 'rep_case_closed')
GROUP BY
    month, r.team
ORDER BY
    month, r.team;
"""

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_nps_por_equipo = pd.read_sql(query_nps_por_equipo, conn)

# Mostrar los resultados
df_nps_por_equipo

,month,team,cantidad_promotores,cantidad_detractores,total_encuestas,nps
0,2020-01,1,1,0,1,100.0
1,2020-02,2,1,0,1,100.0


Agrupación por Mes y Equipo: Se extrae el mes de las fechas de interacción (strftime('%Y-%m', i.int_date)) y se une cada interacción con su representante (r.team).
Cálculo de Promotores y Detractores: Se cuentan los casos con NPS entre 9-10 (promotores) y 0-6 (detractores) de las encuestas..
Uniones y Filtros: Las interacciones relevantes (rep_response, rep_derivation, rep_case_closed) se enlazan con las encuestas y representantes.